In [1]:
!pip install -q transformers[torch]
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("Someman/news_nepali")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15580 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1732 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'article_summary'],
        num_rows: 15580
    })
    test: Dataset({
        features: ['article', 'article_summary'],
        num_rows: 1732
    })
})

In [3]:
dataset = dataset["train"]

In [4]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())

In [5]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [6]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")

[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġpre', (10, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

In [7]:
tokenizer.pre_tokenizer.pre_tokenize_str("२३ दिन हिँडेर काठमाडौं आइपुगे मिटरब्याज पीडित (तस्वीरहरु)")

[('à¥¨à¥©', (0, 2)),
 ('Ġà¤¦', (2, 4)),
 ('à¤¿', (4, 5)),
 ('à¤¨', (5, 6)),
 ('Ġà¤¹', (6, 8)),
 ('à¤¿à¤ģ', (8, 10)),
 ('à¤¡', (10, 11)),
 ('à¥ĩ', (11, 12)),
 ('à¤°', (12, 13)),
 ('Ġà¤ķ', (13, 15)),
 ('à¤¾', (15, 16)),
 ('à¤łà¤®', (16, 18)),
 ('à¤¾', (18, 19)),
 ('à¤¡', (19, 20)),
 ('à¥Įà¤Ĥ', (20, 22)),
 ('Ġà¤Ĩà¤ĩà¤ª', (22, 26)),
 ('à¥ģ', (26, 27)),
 ('à¤Ĺ', (27, 28)),
 ('à¥ĩ', (28, 29)),
 ('Ġà¤®', (29, 31)),
 ('à¤¿', (31, 32)),
 ('à¤Łà¤°à¤¬', (32, 35)),
 ('à¥į', (35, 36)),
 ('à¤¯', (36, 37)),
 ('à¤¾', (37, 38)),
 ('à¤ľ', (38, 39)),
 ('Ġà¤ª', (39, 41)),
 ('à¥Ģ', (41, 42)),
 ('à¤¡', (42, 43)),
 ('à¤¿', (43, 44)),
 ('à¤¤', (44, 45)),
 ('Ġ(', (45, 47)),
 ('à¤¤à¤¸', (47, 49)),
 ('à¥į', (49, 50)),
 ('à¤µ', (50, 51)),
 ('à¥Ģ', (51, 52)),
 ('à¤°à¤¹à¤°', (52, 55)),
 ('à¥ģ)', (55, 57))]

In [8]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset["article"][i : i + 1000]

In [9]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [14]:
sentence = "२३ दिन हिँडेर काठमाडौं आइपुगे मिटरब्याज पीडित (तस्वीरहरु)"
encoding = tokenizer.encode(sentence)
start, end = encoding.offsets[4]
sentence[start:end]

' ह'

In [18]:
encoding

Encoding(num_tokens=38, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [16]:
tokenizer.decoder = decoders.ByteLevel()

In [17]:
tokenizer.decode(encoding.ids)

'२३ दिन हिँडेर काठमाडौं आइपुगे मिटरब्याज पीडित (तस्वीरहरु)'